In [ ]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
auction = pd.read_csv("data/auction.csv")
brand = pd.read_csv("data/brand.csv")
category = pd.read_csv("data/category.csv")
danjobetsu = pd.read_csv("data/danjobetsu.csv")
genre = pd.read_csv("data/genre.csv")
genregroup = pd.read_csv("data/genregroup.csv")
itemshou = pd.read_csv("data/itemshou.csv")
kaiin = pd.read_csv("data/kaiin.csv")
line = pd.read_csv("data/line.csv")
nyuukaoshirase = pd.read_csv("data/nyuuka_oshirase.csv")
rakusatsu = pd.read_csv("data/rakusatsu.csv")
# search_log=pd.read_csv("data/search_log.tsv")
shudounyuusatsu = pd.read_csv("data/shudounyuusatsu.csv")
watchlist = pd.read_csv("data/watchlist.csv")

submission = pd.read_csv("data/sample_submission.csv")
submission = submission.KaiinID.unique()
submission = pd.DataFrame([list(submission)], index=["KaiinID"]).T

auction["CreateDateOrder"] = auction.CreateDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
auction["CreateDateOrder"]=auction.CreateDateOrder.map(lambda x:x.days).map(int)
watchlist["TourokuDateOrder"] = watchlist.TourokuDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
watchlist["TourokuDateOrder"]=watchlist["TourokuDateOrder"].map(lambda x:x.days).map(int)
watchlist = pd.merge(watchlist, auction, on="AuctionID")

shudounyuusatsu["ShudouNyuusatsuDateOrder"] = shudounyuusatsu.ShudouNyuusatsuDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
shudounyuusatsu["ShudouNyuusatsuDateOrder"]=shudounyuusatsu["ShudouNyuusatsuDateOrder"].map(lambda x:x.days).map(int)
shudounyuusatsu = pd.merge(shudounyuusatsu, auction, on='AuctionID')

rakusatsu["RakusatsuDateOrder"]=rakusatsu.RakusatsuDate.map(lambda x: x[0:10]).map(
    lambda x: datetime.strptime(x, '%Y-%m-%d'))-datetime.strptime('2019-10-01', '%Y-%m-%d')
rakusatsu["RakusatsuDateOrder"]=rakusatsu["RakusatsuDateOrder"].map(lambda x:x.days).map(int)
rakusatsu = pd.merge(rakusatsu, submission, on='KaiinID')
rakusatsu = pd.merge(rakusatsu, auction, on='AuctionID')

i=1
dateorder = -7
inputdateorder= dateorder-7*i

In [46]:
def createfeature(inputsubmission, inputwatchlist, inputshudounyuusatsu, inputrakusatsu, inputdateorder):
    df = pd.concat([inputwatchlist[["KaiinID", "ShouhinID"]],
                    inputshudounyuusatsu[["KaiinID", "ShouhinID"]]])
    df = df[df.duplicated() != True]
    df = pd.merge(df, auction[["AuctionID", "ShouhinID"]], on='ShouhinID')
    df = df[df.duplicated() != True]
    df = pd.merge(df, inputsubmission, on="KaiinID")
    auction['CreateDateOrder'] = inputdateorder-auction['CreateDateOrder']
    df = pd.merge(df, auction[["AuctionID", "ShouhinShubetsuID", "ShouhinID", "SaishuppinKaisuu",
                               "ConditionID", "BrandID", "GenreID", "GenreGroupID", "LineID", "ColorID", "DanjobetsuID", ]])
    inputwatchlist.TourokuDateOrder = inputdateorder-inputwatchlist.TourokuDateOrder
    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.max(
    ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuDateOrdermax'}), how='left')
    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.min(
    ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuDateOrdermin'}), how='left')
    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.count(
    ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuCount'}), how='left')
    df["TourokuDatemaxmindiff"] = df.TourokuDateOrdermax-df.TourokuDateOrdermin
    df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).SakujoFlag.max(
    ).reset_index().rename(columns={'SakujoFlag': 'watchSakujo'}), how='left')
    Shouhincount = pd.concat([pd.Series(list(inputwatchlist.groupby("ShouhinID").ShouhinID.count(
    ).index)), pd.Series(inputwatchlist.groupby("ShouhinID").ShouhinID.count().values)], axis=1)
    Shouhincount.columns = ["ShouhinID", "ShouhinWatchedCount"]
    df = pd.merge(df, Shouhincount, how="left")
    Auctioncount = pd.concat([pd.Series(list(inputwatchlist.groupby("AuctionID").AuctionID.count(
    ).index)), pd.Series(inputwatchlist.groupby("AuctionID").AuctionID.count().values)], axis=1)
    Auctioncount.columns = ["AuctionID", "AuctionWatchedCount"]
    Auctioncount
    df = pd.merge(df, Auctioncount, how="left")
    Brandcount = pd.concat([pd.Series(list(inputwatchlist.groupby("BrandID").BrandID.count(
    ).index)), pd.Series(inputwatchlist.groupby("BrandID").BrandID.count().values)], axis=1)
    Brandcount.columns = ["BrandID", "BrandWatchedCount"]
    Brandcount
    df = pd.merge(df, Brandcount, how="left")
    Linecount = pd.concat([pd.Series(list(inputwatchlist.groupby("LineID").LineID.count(
    ).index)), pd.Series(inputwatchlist.groupby("LineID").LineID.count().values)], axis=1)
    Linecount.columns = ["LineID", "LineWatchedCount"]

    df = pd.merge(df, Linecount, how="left")
    ShouhinKaiinWatchcount = inputwatchlist.groupby(
        ["KaiinID", "ShouhinID"]).AuctionID.count()
    ShouhinKaiinWatchcount = ShouhinKaiinWatchcount.reset_index().rename(
        columns={"AuctionID": "ShouhinKaiinWatchCount"})
    df = pd.merge(df, ShouhinKaiinWatchcount, how="left")
    BrandKaiinWatchcount=inputwatchlist.groupby(["KaiinID","BrandID"]).AuctionID.count()
    BrandKaiinWatchcount=BrandKaiinWatchcount.reset_index().rename(columns={"AuctionID":"BrandKaiinWatchCount"})
    df=pd.merge(df,BrandKaiinWatchcount,how="left")
    GenreKaiinWatchcount=inputwatchlist.groupby(["KaiinID","GenreID"]).AuctionID.count()
    GenreKaiinWatchcount=GenreKaiinWatchcount.reset_index().rename(columns={"AuctionID":"GenreKaiinWatchCount"})
    df=pd.merge(df,GenreKaiinWatchcount,how="left")
    GenreGroupKaiinWatchcount=inputwatchlist.groupby(["KaiinID","GenreGroupID"]).AuctionID.count()
    GenreGroupKaiinWatchcount=GenreGroupKaiinWatchcount.reset_index().rename(columns={"AuctionID":"GenreGroupKaiinWatchCount"})
    df=pd.merge(df,GenreGroupKaiinWatchcount,how="left")
    inputshudounyuusatsu.ShudouNyuusatsuDateOrder=inputdateorder-inputshudounyuusatsu.ShudouNyuusatsuDateOrder
    df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.max().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermax'}),how='left')
    df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.min().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermin'}),how='left')
    df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.count().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'CreateCount'}),how='left')
    df["CreateDatemaxmindiff"]=df.ShudouNyuusatsuDateOrdermax-df.ShudouNyuusatsuDateOrdermin
    AuctionNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().index)),pd.Series(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().values)],axis=1)
    AuctionNyuusatsucount.columns=["AuctionID","AuctionNyuudstsuCount"]
    AuctionNyuusatsucount
    df=pd.merge(df,AuctionNyuusatsucount,how="left")
    KaiinNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().index)),pd.Series(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().values)],axis=1)
    KaiinNyuusatsucount.columns=["KaiinID","KaiinNyuudstsuCount"]
    df=pd.merge(df,KaiinNyuusatsucount,how="left")
    BrandNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
    BrandNyuusatsucount.columns=["BrandID","BrandNyuudstsuCount"]
    df=pd.merge(df,BrandNyuusatsucount,how="left")
    BrandNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
    BrandNyuusatsucount.columns=["BrandID","BrandNyuudstsuCount"]
    df=pd.merge(df,BrandNyuusatsucount,how="left")
    
    LineNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("LineID").LineID.count().index)),pd.Series(inputshudounyuusatsu.groupby("LineID").LineID.count().values)],axis=1)
    LineNyuusatsucount.columns=["LineID","LineNyuudstsuCount"]
    df=pd.merge(df,LineNyuusatsucount,how="left")
    
    ShouhinKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","ShouhinID"]).AuctionID.count()
    ShouhinKaiinNyuusatsucount=ShouhinKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"ShouhinKaiinNyuusatsuCount"})
    df=pd.merge(df,ShouhinKaiinNyuusatsucount,how="left")
    
    BrandKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","BrandID"]).AuctionID.count()
    BrandKaiinNyuusatsucount=BrandKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"BrandKaiinNyuusatsuCount"})
    df=pd.merge(df,BrandKaiinNyuusatsucount,how="left")

    GenreKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","GenreID"]).AuctionID.count()
    GenreKaiinNyuusatsucount=GenreKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"GenreKaiinNyuusatuCount"})
    df=pd.merge(df,GenreKaiinNyuusatsucount,how="left")

    LineKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","LineID"]).AuctionID.count()
    LineKaiinNyuusatsucount=LineKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"LineKaiinNyuusatuCount"})
    df=pd.merge(df,LineKaiinNyuusatsucount,how="left")

    df=df.drop(["ShouhinID","ShouhinShubetsuID","BrandID","GenreID","GenreGroupID","LineID"],axis=1)

    df=df.fillna(-1)
    
    return df

In [87]:
train_df=None
i=1
dateorder = -7


for i in range(1,5):
    inputdateorder= dateorder-7*i
    train_watch=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
    train_watch["tag"]=1
    train_watch=train_watch[["KaiinID","AuctionID","tag"]]
    train_watch
    train_nyuusatsu=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
    train_nyuusatsu["tag"]=2
    train_nyuusatsu=train_nyuusatsu[["KaiinID","AuctionID","tag"]]
    train_ref_df=pd.concat([train_nyuusatsu,train_watch]).groupby(["KaiinID","AuctionID"]).max().reset_index()
    train_KaiinID=pd.Series(train_ref_df.KaiinID.unique())
    train_KaiinID.name="KaiinID"
    train_KaiinID=pd.DataFrame(train_KaiinID)
    train_watchlist=watchlist[watchlist.TourokuDateOrder < dateorder-7*i]
    train_shudounyuusatsu=shudounyuusatsu[shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i]
    train_rakusatsu=rakusatsu[rakusatsu.RakusatsuDateOrder<dateorder-7*i]
    watch_kari=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
    watch_kari["watchtag"]=1
    watch_kari=watch_kari[["KaiinID","AuctionID","watchtag"]]
    train_df_child=pd.merge(train_df_child,watch_kari,how="left")
    shudounyuusatsu_kari=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
    shudounyuusatsu_kari["nyuusatsutag"]=1
    shudounyuusatsu_kari=shudounyuusatsu_kari[["KaiinID","AuctionID","nyuusatsutag"]]
    train_df_child=pd.merge(train_df_child,shudounyuusatsu_kari,how="left")
    train_df_child=train_df_child.fillna(0)

    train_df=pd.concat([train_df,train_df_child])

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [14]:
train_nyuusatsu=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
train_nyuusatsu["tag"]=2
train_nyuusatsu=train_nyuusatsu[["KaiinID","AuctionID","tag"]]

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
train_ref_df=pd.concat([train_nyuusatsu,train_watch]).groupby(["KaiinID","AuctionID"]).max().reset_index()

In [43]:
train_KaiinID=pd.Series(train_ref_df.KaiinID.unique())
train_KaiinID.name="KaiinID"
train_KaiinID=pd.DataFrame(train_KaiinID)

In [30]:
train_watchlist=watchlist[watchlist.TourokuDateOrder < dateorder-7*i]


In [33]:
train_shudounyuusatsu=shudounyuusatsu[shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i]


In [39]:
train_rakusatsu=rakusatsu[rakusatsu.RakusatsuDateOrder<dateorder-7*i]

In [55]:
i

1

In [73]:
# train_df_child=createfeature(train_KaiinID, train_watchlist, train_shudounyuusatsu, train_rakusatsu, dateorder-7*i)
watch_kari=watchlist[(watchlist.TourokuDateOrder>=dateorder-7*i)&(watchlist.TourokuDateOrder<dateorder-7*i+7)]
watch_kari["watchtag"]=1
watch_kari=watch_kari[["KaiinID","AuctionID","watchtag"]]
train_df_child=pd.merge(train_df_child,watch_kari,how="left")

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
shudounyuusatsu_kari=shudounyuusatsu[(shudounyuusatsu.ShudouNyuusatsuDateOrder>=dateorder-7*i)&(shudounyuusatsu.ShudouNyuusatsuDateOrder<dateorder-7*i+7)]
shudounyuusatsu_kari["nyuusatsutag"]=1
shudounyuusatsu_kari=shudounyuusatsu_kari[["KaiinID","AuctionID","nyuusatsutag"]]
train_df_child=pd.merge(train_df_child,shudounyuusatsu_kari,how="left")

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [84]:
train_df_child=train_df_child.fillna(0)


,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,AuctionNyuudstsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag,nyuusatsutag
0,231713,1036190,0,6,15,0,-395.0,-395.0,1.0,0.0,...,-1.0,45.0,319.0,572160.0,-1.0,-1.0,-1.0,41.0,0.0,0.0
1,231713,4106265,0,6,68,0,-367.0,-367.0,1.0,0.0,...,-1.0,45.0,4521.0,572160.0,-1.0,-1.0,-1.0,41.0,0.0,0.0
2,58564,4106265,0,6,68,0,-372.0,-372.0,1.0,0.0,...,-1.0,54.0,4521.0,572160.0,-1.0,-1.0,1.0,54.0,0.0,0.0
3,81233,4106265,0,6,68,0,-395.0,-395.0,1.0,0.0,...,-1.0,47.0,4521.0,572160.0,-1.0,-1.0,-1.0,47.0,0.0,0.0
4,79459,4106265,0,6,68,0,-362.0,-362.0,1.0,0.0,...,-1.0,25.0,4521.0,572160.0,-1.0,7.0,-1.0,25.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461870,47024,2007419,0,6,297,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,421.0,572160.0,1.0,1.0,1.0,1.0,0.0,1.0
3461871,124155,1784243,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,182.0,572160.0,1.0,1.0,1.0,1.0,0.0,1.0
3461872,87578,2923626,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,0.0,0.0
3461873,87578,1977247,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,0.0,0.0


In [86]:
train_df=None
train_df=pd.concat([train_df,train_df_child])

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,AuctionNyuudstsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreKaiinNyuusatuCount,LineKaiinNyuusatuCount,watchtag,nyuusatsutag
0,231713,1036190,0,6,15,0,-395.0,-395.0,1.0,0.0,...,-1.0,45.0,319.0,572160.0,-1.0,-1.0,-1.0,41.0,NaN,NaN
1,231713,4106265,0,6,68,0,-367.0,-367.0,1.0,0.0,...,-1.0,45.0,4521.0,572160.0,-1.0,-1.0,-1.0,41.0,NaN,NaN
2,58564,4106265,0,6,68,0,-372.0,-372.0,1.0,0.0,...,-1.0,54.0,4521.0,572160.0,-1.0,-1.0,1.0,54.0,NaN,NaN
3,81233,4106265,0,6,68,0,-395.0,-395.0,1.0,0.0,...,-1.0,47.0,4521.0,572160.0,-1.0,-1.0,-1.0,47.0,NaN,NaN
4,79459,4106265,0,6,68,0,-362.0,-362.0,1.0,0.0,...,-1.0,25.0,4521.0,572160.0,-1.0,7.0,-1.0,25.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461870,47024,2007419,0,6,297,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,421.0,572160.0,1.0,1.0,1.0,1.0,NaN,1.0
3461871,124155,1784243,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,182.0,572160.0,1.0,1.0,1.0,1.0,NaN,1.0
3461872,87578,2923626,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,NaN,NaN
3461873,87578,1977247,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0,NaN,NaN


In [59]:
# inputsubmission, inputwatchlist, inputshudounyuusatsu, inputrakusatsu, inputdateorder=(train_KaiinID, train_watchlist, train_shudounyuusatsu, train_rakusatsu, dateorder-7*i)
# df = pd.concat([inputwatchlist[["KaiinID", "ShouhinID"]],
#                 inputshudounyuusatsu[["KaiinID", "ShouhinID"]]])
# df = df[df.duplicated() != True]
# df = pd.merge(df, auction[["AuctionID", "ShouhinID"]], on='ShouhinID')
# df = df[df.duplicated() != True]
# df = pd.merge(df, inputsubmission, on="KaiinID")
# auction['CreateDateOrder'] = inputdateorder-auction['CreateDateOrder']
# df = pd.merge(df, auction[["AuctionID", "ShouhinShubetsuID", "ShouhinID", "SaishuppinKaisuu",
#                            "ConditionID", "BrandID", "GenreID", "GenreGroupID", "LineID", "ColorID", "DanjobetsuID", ]])
# inputwatchlist.TourokuDateOrder = inputdateorder-inputwatchlist.TourokuDateOrder
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.max(
# ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuDateOrdermax'}), how='left')
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.min(
# ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuDateOrdermin'}), how='left')
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).TourokuDateOrder.count(
# ).reset_index().rename(columns={'TourokuDateOrder': 'TourokuCount'}), how='left')
# df["TourokuDatemaxmindiff"] = df.TourokuDateOrdermax-df.TourokuDateOrdermin
# df = pd.merge(df, inputwatchlist.groupby(["KaiinID", "AuctionID"]).SakujoFlag.max(
# ).reset_index().rename(columns={'SakujoFlag': 'watchSakujo'}), how='left')
# Shouhincount = pd.concat([pd.Series(list(inputwatchlist.groupby("ShouhinID").ShouhinID.count(
# ).index)), pd.Series(inputwatchlist.groupby("ShouhinID").ShouhinID.count().values)], axis=1)
# Shouhincount.columns = ["ShouhinID", "ShouhinWatchedCount"]
# df = pd.merge(df, Shouhincount, how="left")
# Auctioncount = pd.concat([pd.Series(list(inputwatchlist.groupby("AuctionID").AuctionID.count(
# ).index)), pd.Series(inputwatchlist.groupby("AuctionID").AuctionID.count().values)], axis=1)
# Auctioncount.columns = ["AuctionID", "AuctionWatchedCount"]
# Auctioncount
# df = pd.merge(df, Auctioncount, how="left")
# Brandcount = pd.concat([pd.Series(list(inputwatchlist.groupby("BrandID").BrandID.count(
# ).index)), pd.Series(inputwatchlist.groupby("BrandID").BrandID.count().values)], axis=1)
# Brandcount.columns = ["BrandID", "BrandWatchedCount"]
# Brandcount
# df = pd.merge(df, Brandcount, how="left")
# Linecount = pd.concat([pd.Series(list(inputwatchlist.groupby("LineID").LineID.count(
# ).index)), pd.Series(inputwatchlist.groupby("LineID").LineID.count().values)], axis=1)
# Linecount.columns = ["LineID", "LineWatchedCount"]

# df = pd.merge(df, Linecount, how="left")
# ShouhinKaiinWatchcount = inputwatchlist.groupby(
#     ["KaiinID", "ShouhinID"]).AuctionID.count()
# ShouhinKaiinWatchcount = ShouhinKaiinWatchcount.reset_index().rename(
#     columns={"AuctionID": "ShouhinKaiinWatchCount"})
# df = pd.merge(df, ShouhinKaiinWatchcount, how="left")
# BrandKaiinWatchcount=inputwatchlist.groupby(["KaiinID","BrandID"]).AuctionID.count()
# BrandKaiinWatchcount=BrandKaiinWatchcount.reset_index().rename(columns={"AuctionID":"BrandKaiinWatchCount"})
# df=pd.merge(df,BrandKaiinWatchcount,how="left")
# GenreKaiinWatchcount=inputwatchlist.groupby(["KaiinID","GenreID"]).AuctionID.count()
# GenreKaiinWatchcount=GenreKaiinWatchcount.reset_index().rename(columns={"AuctionID":"GenreKaiinWatchCount"})
# df=pd.merge(df,GenreKaiinWatchcount,how="left")
# GenreGroupKaiinWatchcount=inputwatchlist.groupby(["KaiinID","GenreGroupID"]).AuctionID.count()
# GenreGroupKaiinWatchcount=GenreGroupKaiinWatchcount.reset_index().rename(columns={"AuctionID":"GenreGroupKaiinWatchCount"})
# df=pd.merge(df,GenreGroupKaiinWatchcount,how="left")
# inputshudounyuusatsu.ShudouNyuusatsuDateOrder=inputdateorder-inputshudounyuusatsu.ShudouNyuusatsuDateOrder
# df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.max().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermax'}),how='left')
# df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.min().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'ShudouNyuusatsuDateOrdermin'}),how='left')
# df=pd.merge(df,inputshudounyuusatsu.groupby(["KaiinID","AuctionID"]).ShudouNyuusatsuDateOrder.count().reset_index().rename(columns={'ShudouNyuusatsuDateOrder': 'CreateCount'}),how='left')
# df["CreateDatemaxmindiff"]=df.ShudouNyuusatsuDateOrdermax-df.ShudouNyuusatsuDateOrdermin
# AuctionNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().index)),pd.Series(inputshudounyuusatsu.groupby("AuctionID").AuctionID.count().values)],axis=1)
# AuctionNyuusatsucount.columns=["AuctionID","AuctionNyuudstsuCount"]
# AuctionNyuusatsucount
# df=pd.merge(df,AuctionNyuusatsucount,how="left")
# KaiinNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().index)),pd.Series(inputshudounyuusatsu.groupby("KaiinID").KaiinID.count().values)],axis=1)
# KaiinNyuusatsucount.columns=["KaiinID","KaiinNyuudstsuCount"]
# df=pd.merge(df,KaiinNyuusatsucount,how="left")
# BrandNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
# BrandNyuusatsucount.columns=["BrandID","BrandNyuudstsuCount"]
# df=pd.merge(df,BrandNyuusatsucount,how="left")
# BrandNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("BrandID").BrandID.count().index)),pd.Series(inputshudounyuusatsu.groupby("BrandID").BrandID.count().values)],axis=1)
# BrandNyuusatsucount.columns=["BrandID","BrandNyuudstsuCount"]
# df=pd.merge(df,BrandNyuusatsucount,how="left")

# LineNyuusatsucount=pd.concat([pd.Series(list(inputshudounyuusatsu.groupby("LineID").LineID.count().index)),pd.Series(inputshudounyuusatsu.groupby("LineID").LineID.count().values)],axis=1)
# LineNyuusatsucount.columns=["LineID","LineNyuudstsuCount"]
# df=pd.merge(df,LineNyuusatsucount,how="left")

# ShouhinKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","ShouhinID"]).AuctionID.count()
# ShouhinKaiinNyuusatsucount=ShouhinKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"ShouhinKaiinNyuusatsuCount"})
# df=pd.merge(df,ShouhinKaiinNyuusatsucount,how="left")

# BrandKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","BrandID"]).AuctionID.count()
# BrandKaiinNyuusatsucount=BrandKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"BrandKaiinNyuusatsuCount"})
# df=pd.merge(df,BrandKaiinNyuusatsucount,how="left")

# GenreKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","GenreID"]).AuctionID.count()
# GenreKaiinNyuusatsucount=GenreKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"GenreKaiinNyuusatuCount"})
# df=pd.merge(df,GenreKaiinNyuusatsucount,how="left")

# LineKaiinNyuusatsucount=inputshudounyuusatsu.groupby(["KaiinID","LineID"]).AuctionID.count()
# LineKaiinNyuusatsucount=LineKaiinNyuusatsucount.reset_index().rename(columns={"AuctionID":"LineKaiinNyuusatuCount"})
# df=pd.merge(df,LineKaiinNyuusatsucount,how="left")

# df=df.drop(["ShouhinID","ShouhinShubetsuID","BrandID","GenreID","GenreGroupID","LineID"],axis=1)

# df=df.fillna(-1)

# df

/home/ogata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,ColorID,DanjobetsuID,TourokuDateOrdermax,TourokuDateOrdermin,TourokuCount,TourokuDatemaxmindiff,...,CreateCount,CreateDatemaxmindiff,AuctionNyuudstsuCount,KaiinNyuudstsuCount,BrandNyuudstsuCount,LineNyuudstsuCount,ShouhinKaiinNyuusatsuCount,BrandKaiinNyuusatsuCount,GenreKaiinNyuusatuCount,LineKaiinNyuusatuCount
0,231713,1036190,0,6,15,0,381.0,381.0,1.0,0.0,...,-1.0,-1.0,-1.0,45.0,319.0,572160.0,-1.0,-1.0,-1.0,41.0
1,231713,4106265,0,6,68,0,353.0,353.0,1.0,0.0,...,-1.0,-1.0,-1.0,45.0,4521.0,572160.0,-1.0,-1.0,-1.0,41.0
2,58564,4106265,0,6,68,0,358.0,358.0,1.0,0.0,...,-1.0,-1.0,-1.0,54.0,4521.0,572160.0,-1.0,-1.0,1.0,54.0
3,81233,4106265,0,6,68,0,381.0,381.0,1.0,0.0,...,-1.0,-1.0,-1.0,47.0,4521.0,572160.0,-1.0,-1.0,-1.0,47.0
4,79459,4106265,0,6,68,0,348.0,348.0,1.0,0.0,...,-1.0,-1.0,-1.0,25.0,4521.0,572160.0,-1.0,7.0,-1.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459604,47024,2007419,0,6,297,0,-1.0,-1.0,-1.0,-1.0,...,1.0,0.0,1.0,1.0,421.0,572160.0,1.0,1.0,1.0,1.0
3459605,124155,1784243,0,6,0,0,-1.0,-1.0,-1.0,-1.0,...,1.0,0.0,1.0,1.0,182.0,572160.0,1.0,1.0,1.0,1.0
3459606,87578,2923626,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0
3459607,87578,1977247,0,6,68,2,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,1.0,45.0,572160.0,1.0,1.0,1.0,1.0
